**Lets create some code 💪🔑🌐**

Read data and add to dataframe

Shafts-data comes from htlm file of http://www.zechenkarte.de/wiki/index.php?title=Gesamtkarte

Mine data comes from the xlsx file, where the data starts from line 5.

In [62]:
!pip install fuzz
!pip install fuzzywuzzy
!pip install pandas as pd
!pip install json
!pip install csv
!pip install rdflib
!pip install folium
!pip install matplotlib
!pip install mapclassify

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [63]:
import pandas as pd
import openpyxl
from fuzzywuzzy import fuzz
from pandas import json_normalize
import json


from rdflib import Graph, Literal, Namespace, RDF, URIRef
from rdflib.namespace import GEO, XSD

In [64]:
# Import data form the excel sheet

file_path = 'C:/Users/rohne/Documents/LDAC/LDAC2024/data/Steinkohlenzechen_NRW.xlsx'  # Adjust the path as needed
excel_file = pd.ExcelFile(file_path)

# Iterate through all sheet names
#sheets_data ={}
dataframe = []
for sheet_name in excel_file.sheet_names:
    # Read the sheet starting from the 5th line (index 4) with the 5th line as header (index 4)
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=4)
    df['year']=sheet_name
    dataframe.append(df)

mines = pd.concat(dataframe, ignore_index=True)

    # Store the DataFrame in the dictionary
    #sheets_data[sheet_name] = shafts

In [65]:
#Read data from the html file

shafts_path = 'Mines_all_small.json'
with open(shafts_path) as f:
    data = json.load(f)
shafts = json_normalize(data['locations'])

# print(mines.Mine)
# print(shafts.title)

In [66]:
#Data preparation, Add additional Table for fuzzy 
shafts['title_fuzzy']=shafts['title']
shafts['title_fuzzy'] = shafts['title_fuzzy'].str.replace(' Schacht', '', case=False)
shafts['title_fuzzy'] = shafts['title_fuzzy'].str.strip()

mines['title_fuzzy']=mines['Name des Bergwerks']
words_to_remove = ['Gewerkschaft', 'Zeche', 'ver.', 'Bergwerk', 'Bergbau', 'Aktien', 'Gesellschaft']
pattern = '|'.join(words_to_remove)
mines['title_fuzzy'] = mines['title_fuzzy'].str.replace(pattern, '', case=False, regex=True)
mines['title_fuzzy'] = mines['title_fuzzy'].str.strip()

In [67]:
# Function to compute similarity and group mines
def group_similar_titles(mines, threshold=85):
    # List to store group indices
    groups = [-1] * len(mines)
    group_id = 0
    
    # Iterate over each title to compare it with others
    for i in range(len(mines)):
        if groups[i] == -1:  # If not already assigned to a group
            groups[i] = group_id
            for j in range(i + 1, len(mines)):
                if groups[j] == -1:  # If not already assigned to a group
                    similarity_score = fuzz.ratio(str(mines.loc[i, 'title_fuzzy']+ ' '+mines.loc[i,'Ort']),str(mines.loc[j, 'title_fuzzy']+ ' '+mines.loc[j,'Ort']))
                
                    if similarity_score >= threshold:
                        groups[j] = group_id
                        #print(mines.loc[i, 'title_fuzzy'],',', mines.loc[j, 'title_fuzzy'],'->', similarity_score)
            group_id += 1
    
    return groups

# Apply the function to group similar titles
mines['mine_group'] = group_similar_titles(mines)

In [68]:
# Define a function to concatenate unique values while handling NaNs
def concat_unique(series):
    unique_values = series.dropna().unique()  # Drop NaNs and get unique values
    return ', '.join(unique_values) if unique_values.size > 0 else None

mines_only = mines.copy()
mines_only = mines_only.groupby('mine_group').agg(
    title = ('Name des Bergwerks', 'first')
    ,ort = ('Ort','first')
    ,revier = ('Revier','first')
    ,title_fuzzy = ('title_fuzzy','first')
    ,weitere_informationen = ('weitere Informationen',concat_unique)  # Additional information

    #company_form =('Unternehmensform','first')
    ).reset_index()

In [69]:
# Define a function to compute string similarity
def compute_similarity(string1, string2):
    return fuzz.ratio(string1, string2)  # You can use other similarity metrics here

matches = []
for index1, row1 in mines_only.iterrows():
    for index2, row2 in shafts.iterrows():
        similarity_score = compute_similarity(row1['title_fuzzy'], row2['title_fuzzy']) 
        if similarity_score >= 60:  # Set a threshold for similarity
            matches.append((index1, index2, similarity_score))

matches_df = pd.DataFrame(matches, columns=['index_mines_group', 'index_shafts', 'similarity_score'])
           
# Output matches
if len(matches) == 0:
    print("No matches found.")
else:
    print("Matches found:", len(matches))

Matches found: 1018


In [72]:
# Create ttl file for mines 
#
EX = Namespace("http://example.org/")
#GEO = Namespace("http://www.geonames.org/ontology#")
SCHEMA = Namespace("http://schema.org/")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")

# Create an RDF graph
g = Graph()
g.remove((None, None, None))
g.bind("ex", EX)
g.bind("geo", GEO)
g.bind("schema", SCHEMA)

for index, row in mines_only.iterrows():

    title = row.title
    ort = row.ort
    revier = row.revier
    UID_mine = row.mine_group
    weitere_informationen = row.weitere_informationen

    mine_uri = URIRef(EX + "mine/"+ str(UID_mine))
    
    g.add((mine_uri, RDF.type, EX.Mine))
    g.add((mine_uri, EX.name, Literal(title)))
    g.add((mine_uri, GEO.location, Literal(ort)))
    g.add((mine_uri, SCHEMA.Place, Literal(revier)))
    g.add((mine_uri, EX.info, Literal(weitere_informationen)))
    
ttl_file = 'mines.ttl'
g.serialize(destination=ttl_file, format='turtle')

<Graph identifier=N6e5d8440326441d38faaa92b56faafcb (<class 'rdflib.graph.Graph'>)>

In [73]:
materials = mines.copy()
materials['Koksproduktion'] = materials['Koksproduktion in Tonnen (1906)'].fillna(materials['Koksproduktion in Tonnen'])
materials['Förderung'] = materials['Förderung in Tonnen (1906)'].fillna(materials['Förderung in Tonnen'])

In [104]:

# Materials
EX = Namespace("http://example.org/")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")
SCHEMA = Namespace("http://schema.org/")

# Create an RDF graph
g = Graph()
g.remove((None, None, None))
g.bind("ex", EX)
g.bind("schema", SCHEMA)

for index, row in materials.iterrows():
    title = row['abgebauter Rohstoff']  # Material name
    #ort = row['Ort']  # Location
    #revier = row['Revier']  # Mining district
    unternehmensform = row['Unternehmensform']  # Company type
    syndikatsbeteiligung = row['Syndikatsbeteiligung']  # Syndicate participation
    #foerderung_1906 = row['Förderung in Tonnen (1906)']  # Production in tons (1906)
    kohlensorte = row['Kohlensorte']  # Coal type
    kokssorte = row['Kokssorte']  # Coke type
    #koksproduktion_1906 = row['Koksproduktion in Tonnen (1906)']  # Coke production in tons (1906)
    sonstige_nebenprodukte = row['sonstige Nebenprodukte']  # Other by-products
    #seite = row['Seite']  # Page number
    year = row['year']  # Year
    foerderung = row['Förderung']  # Production in tons
    koksproduktion = row['Koksproduktion']  # Coke production in tons
    UID_mine = row.mine_group
    mine_uri = URIRef(EX + "mine/"+ str(UID_mine))

    # Create URI for the material
    material_uri = URIRef(EX +"material/" + str(row['mine_group']) + "/" + str(row['year']) ) # Using material name as URI, adjust as needed

    # Add triples to the RDF graph
    g.add((material_uri, RDF.type, EX.Material))
    g.add((material_uri, EX.name, Literal(title)))
    #g.add((material_uri, GEO.name, Literal(ort)))
    #g.add((material_uri, SCHEMA.Place, Literal(revier)))
    g.add((material_uri, EX.Unternehmensform, Literal(unternehmensform)))
    g.add((material_uri, EX.Syndikatsbeteiligung, Literal(syndikatsbeteiligung)))
    g.add((material_uri, EX.Kohlensorte, Literal(kohlensorte)))
    g.add((material_uri, EX.Kokssorte, Literal(kokssorte)))
    g.add((material_uri, EX.sonstige_Nebenprodukte, Literal(sonstige_nebenprodukte)))
    #g.add((material_uri, EX.Seite, Literal(seite)))
    g.add((material_uri, SCHEMA.year, Literal(year,datatype=XSD.gYear)))
    g.add((material_uri, EX.foerderung, Literal(foerderung)))
    g.add((material_uri, EX.koksproduktion, Literal(koksproduktion)))
    g.add((mine_uri, EX.hasMaterial, material_uri))
    g.add((material_uri, EX.hasMine, mine_uri))

ttl_file = 'mines_materials.ttl'
g.serialize(destination=ttl_file, format='turtle')



<Graph identifier=N6eb50e93b1fc47b69f4f269ac665fd5c (<class 'rdflib.graph.Graph'>)>

In [91]:
# Create ttl file for shafts
# Define namespaces
EX = Namespace("http://example.org/")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")

# Create an RDF graph
g = Graph()
g.remove((None, None, None))
g.bind("ex", EX)
g.bind("geo", GEO)

# Process each entry in the JSON data
for index, row in shafts.iterrows():

    title_item = row.title
    latitude = row.lat
    longitude = row.lon
    
    # Create URI for the title and geometry
    title_uri = EX["http://www.zechenkarte.de/wiki/index.php?title=" + title_item.replace(" ", "_")]
    geom_uri = EX[title_item.replace(" ", "_") + "Geom"]
    
    # Add RDF triples to the graph
    g.add((title_uri, RDF.type, EX.Schacht))
    g.add((title_uri, EX.name, Literal(title_item)))
    g.add((title_uri, GEO.hasGeometry, geom_uri))
    
    g.add((geom_uri, RDF.type, GEO.Geometry))
    g.add((geom_uri, GEO.asWKT, Literal(f"POINT({longitude} {latitude})", datatype=GEO.wktLiteral)))

    matches_filtered = matches_df[matches_df.index_shafts == index]
    if not matches_filtered.empty:
        for _, match_row in matches_filtered.iterrows(): 
            UID_mine = match_row["index_mines_group"]
        
            mine_uri = URIRef(EX + "mine/"+ str(UID_mine))


            g.add((mine_uri, EX.hasShafts, title_uri))
            g.add((title_uri, EX.hasMine, mine_uri))

# Serialize the graph to a TTL file
g.serialize(destination='shafts.ttl', format='turtle')

#print("TTL file created successfully.")


<Graph identifier=N6bb6351328d641388d0c46dd5fdbcdb1 (<class 'rdflib.graph.Graph'>)>

In [100]:
##ToDo Add the queries 
import geopandas as gpd
from shapely.geometry import Point
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, GEO

# Define namespaces
EX = Namespace("http://example.org/")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")

# Load the RDF graph from the TTL file
g = Graph()
g.parse("shafts.ttl", format="turtle")
#g.parse("mines.ttl", format="turtle")

# Define the SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?schacht ?name ?wkt
WHERE {
  ?schacht a ex:Schacht ;
           ex:name ?name ;
           ex:
           geo:hasGeometry ?geom .
  ?geom geo:asWKT ?wkt .
}
"""

# Execute the query
results = g.query(query)

In [101]:
print(results)

In [106]:
# Prepare data for GeoDataFrame
data = []
for row in results:
    schacht_uri = str(row.schacht)
    name = str(row.name).strip()
    wkt = str(row.wkt)
    
    # Convert WKT to shapely geometry
    geometry = Point(float(wkt.split('(')[1].split()[0]), float(wkt.split('(')[1].split()[1][:-1]))
    
    data.append({"schacht_uri": schacht_uri, "name": name, "geometry": geometry})

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry="geometry")

# Save the GeoDataFrame to a shapefile
gdf.to_file("schachts.shp", driver='ESRI Shapefile')
gdf.to_file('schachts.geojson', driver='GeoJSON')  

print("Data extraction and shapefile creation complete.")


C:\Users\rohne\AppData\Local\Temp\ipykernel_39400\3579149396.py:17: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("schachts.shp", driver='ESRI Shapefile')


Data extraction and shapefile creation complete.


In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt

# Read the specified sheet (Sheet 2) from the Excel file into a DataFrame
df = pd.read_excel("Result1_Zeche-Shaft-point.xlsx", sheet_name='Sheet2')  # You can also use sheet_name=1 to refer to the second sheet by index

df['POINT'] = df['POINT'].str.strip('"')

# Convert the 'POINT' column from string to Shapely Point objects
df['geometry'] = df['POINT'].apply(wkt.loads)  # Replace 'POINT' with the actual column name if different

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Use the explore method to visualize the data
# Color the points based on the 'Footprint' column (replace 'Footprint' with the actual column name)
#gdf.explore(column='Zeche', cmap='viridis', legend=True)
gdf.explore()
# Set the coordinate reference system (CRS) to
gdf.to_file('Results1_Zeche-Shaft-point_gdf.shp')

C:\Users\rohne\AppData\Local\Temp\ipykernel_29560\923309337.py:25: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file('Results1_Zeche-Shaft-point_gdf.shp')


In [15]:
Zeche_gdf_data = gpd.read_file('Zeche_polygon.geojson')
#geom = str(Zeche_gdf.geometry)
Zeche_data = pd.read_excel("Result2_Zeche_Year_Material.xlsx")

# Extract the geometry
geometry = Zeche_gdf_data.geometry.iloc[0]

# Create a new DataFrame with the desired data from Zeche_data
new_data = Zeche_data.copy()

# Assign the geometry to each row in the new DataFrame
new_data['geometry'] = geometry

# Create a new GeoDataFrame with the combined data and geometry
Zeche_gdf = gpd.GeoDataFrame(new_data, geometry='geometry')

# Now combined_gdf contains both the data from Zeche_data and the geometry from Zeche_gdf
Zeche_gdf.to_file('Zeche_gdf.geojson')

In [14]:
Zeche_gdf

,coal,year,CoalName,CoalWeight,coke,CokeMaterialName,CokeWeight,materialinst,raw,RawMaterialName,RawMaterialWeight,Year,Zeche,geometry
0,_:bc_0_n3-37,1895,"""Fett-, Mager- und Esskohlen""",219950,_:bc_0_n3-38,"""Hochofen- und Gießereikoks""","""82300""",<http://linkedbuildingdata.net/mine/MaterialSe...,_:bc_0_n3-36,"""Steinkohle""","""0""",<https://w3id.org/cmo#1895>,<http://linkedbuildingdata.net/mine/Maria-Anna...,"MULTIPOLYGON (((7.15147 51.44793, 7.15659 51.4..."
1,_:bc_0_n3-40,1990,"""Fett-, Mager- und Esskohlen""",228576,_:bc_0_n3-41,"""Hochofen- und Gießereikoks""","""69142""",<http://linkedbuildingdata.net/mine/MaterialSe...,_:bc_0_n3-39,"""Steinkohle""","""0""",<https://w3id.org/cmo#1990>,<http://linkedbuildingdata.net/mine/Maria-Anna...,"MULTIPOLYGON (((7.15147 51.44793, 7.15659 51.4..."
